In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.neighbors import NearestNeighbors
from econml.dml import LinearDML
import warnings

# Suppress warnings for cleaner output (e.g., convergence warnings in high-dim logistic reg)
warnings.filterwarnings('ignore')

In [16]:
class CausalEstimators:

# A helper class to run various causal inference methods:
# 1. Naive (Unadjusted Difference in Means)
# 2. Regression Adjustment (Linear/Logistic)
# 3. IPTW (Inverse Probability of Treatment Weighting)
# 4. PSM (Propensity Score Matching 1:1)
# 5. DML (Double Machine Learning)

    def __init__(self, X, T, Y, outcome_type= 'continuous'):
        self.X = X
        self.T = T
        self.Y = Y
        self.outcome_type = outcome_type
        # Create a DataFrame for easier handling in naive methods
        self.df = pd.DataFrame(X)
        self.df['T'] = T
        self.df['Y'] = Y
    def run_naive(self):
    
        treated = self.df[self.df['T'] == 1]['Y'].mean()
        control = self.df[self.df['T'] == 0]['Y'].mean()
        return treated - control

    def run_regression_adjustment(self):

        # Combine T and X into one feature matrix
        features = np.column_stack((self.T, self.X))
        model = LinearRegression()
        model.fit(features, self.Y)
        # The coefficient of T (index 0) is the additive effect
        return model.coef_[0]

    def run_iptw(self):
        # Estimate Propensity Scores
        ps_model = LogisticRegression(solver='liblinear', max_iter=2000)
        ps_model.fit(self.X, self.T)
        ps = ps_model.predict_proba(self.X)[:, 1]
        
        # Clip to prevent division by zero or extreme weights (common in small N)
        ps = np.clip(ps, 0.05, 0.95)
        # Calculate weights: 1/PS for treated, 1/(1-PS) for control
        weights = np.where(self.T == 1, 1/ps, 1/(1-ps))
        model = LinearRegression()
        model.fit(self.T.reshape(-1, 1), self.Y, sample_weight=weights)
        return model.coef_[0]

    def run_psm(self):
        # Estimate Propensity Scores
        ps_model = LogisticRegression(solver='liblinear', max_iter=2000)
        ps_model.fit(self.X, self.T)
        ps = ps_model.predict_proba(self.X)[:, 1]
        
        treated_idx = np.where(self.T == 1)[0]
        control_idx = np.where(self.T == 0)[0]
        
        # Safety check for separation
        if len(control_idx) == 0 or len(treated_idx) == 0:
            return np.nan
        # Match each treated unit to nearest control unit based on PS
        nbrs = NearestNeighbors(n_neighbors=1).fit(ps[control_idx].reshape(-1, 1))
        distances, indices = nbrs.kneighbors(ps[treated_idx].reshape(-1, 1))
        matched_control_idx = control_idx[indices.flatten()]
        
        # Difference in Means of matched pairs (Risk Difference for binary)
        return np.mean(self.Y[treated_idx]) - np.mean(self.Y[matched_control_idx])
    
    def run_dml(self):
        # Use shallow trees (depth 2) to prevent overfitting on small N=500
        y_model = GradientBoostingRegressor(n_estimators=50, max_depth=2,
        random_state=42)
        t_model = GradientBoostingClassifier(n_estimators=50, max_depth=2,
        random_state=42)
    
        # Set cv=3 for small sample size (5-fold might be too thin)
        est = LinearDML(model_y=y_model,
                        model_t=t_model,
                        discrete_treatment=True,
                        cv=3,
                        random_state=42)
        est.fit(self.Y, self.T, X=self.X)
        # Return Average Treatment Effect
        return est.effect(self.X).mean()

In [5]:
# Define Method 1 Naive estimates
def run_naive(self):
    
    treated = self.df[self.df['T'] == 1]['Y'].mean()
    control = self.df[self.df['T'] == 0]['Y'].mean()
    return treated - control

In [6]:
# Define Method 2 Linear regression with covariates adjusted
def run_regression_adjustment(self):

    # Combine T and X into one feature matrix
    features = np.column_stack((self.T, self.X))
    model = LinearRegression()
    model.fit(features, self.Y)
    # The coefficient of T (index 0) is the additive effect
    return model.coef_[0]

In [7]:
# Define Method 3 IPTW (Logistic PS) 
def run_iptw(self):
# Estimate Propensity Scores
    ps_model = LogisticRegression(solver='liblinear', max_iter=2000)
    ps_model.fit(self.X, self.T)
    ps = ps_model.predict_proba(self.X)[:, 1]
    
    # Clip to prevent division by zero or extreme weights (common in small N)
    ps = np.clip(ps, 0.05, 0.95)
    # Calculate weights: 1/PS for treated, 1/(1-PS) for control
    weights = np.where(self.T == 1, 1/ps, 1/(1-ps))
    model = LinearRegression()
    model.fit(self.T.reshape(-1, 1), self.Y, sample_weight=weights)
    return model.coef_[0]


In [8]:
# Define Method 4 PSM (1:1 Nearest Neighbor) 
def run_psm(self):
    # Estimate Propensity Scores
    ps_model = LogisticRegression(solver='liblinear', max_iter=2000)
    ps_model.fit(self.X, self.T)
    ps = ps_model.predict_proba(self.X)[:, 1]
    
    treated_idx = np.where(self.T == 1)[0]
    control_idx = np.where(self.T == 0)[0]
    
    # Safety check for separation
    if len(control_idx) == 0 or len(treated_idx) == 0:
        return np.nan
    # Match each treated unit to nearest control unit based on PS
    nbrs = NearestNeighbors(n_neighbors=1).fit(ps[control_idx].reshape(-1, 1))
    distances, indices = nbrs.kneighbors(ps[treated_idx].reshape(-1, 1))
    matched_control_idx = control_idx[indices.flatten()]
    
    # Difference in Means of matched pairs (Risk Difference for binary)
    return np.mean(self.Y[treated_idx]) - np.mean(self.Y[matched_control_idx])

In [9]:
# Define Method 5 DML (Gradient Boosting) 
def run_dml(self):
# Use shallow trees (depth 2) to prevent overfitting on small N=500
    y_model = GradientBoostingRegressor(n_estimators=50, max_depth=2,
    random_state=42)
    t_model = GradientBoostingClassifier(n_estimators=50, max_depth=2,
    random_state=42)

# Set cv=3 for small sample size (5-fold might be too thin)
    est = LinearDML(model_y=y_model,
                    model_t=t_model,
                    discrete_treatment=True,
                    cv=3,
                    random_state=42)
    est.fit(self.Y, self.T, X=self.X)
    # Return Average Treatment Effect
    return est.effect(self.X).mean()

In [10]:
# Data generation function
def generate_rare_disease_data(n=500, p=100, setting='continuous',seed=123):
    np.random.seed(seed)
    
    # 1. Generate Covariates (100 variables)
    X = np.random.normal(0, 1, size=(n, p))
    
    # 2. Define the &quot;True&quot; Confounding Mechanism (The Nuisance Function)
    # Only the first 5 variables (indices 0-4) actually matter.
    # Interaction: X0 * X1
    # Non-linear: X2 squared
    # Linear: X3, X4
    nuisance_term = 0.5 * X[:,0] * X[:,1] + 0.4 * (X[:,2]**2) + 0.3 * X[:,3] + 0.2 * X[:,4]
    # 3. Treatment Assignment (Propensity)
    # P(T=1 | X) depends on the nuisance term
    logit_p = -0.5 + nuisance_term
    prob_t = 1 / (1 + np.exp(-logit_p))
    T = np.random.binomial(1, prob_t)
    TRUE_EFFECT = 1000.0 # True Causal Effect ($1000)
    
    # Outcome depends on T, the nuisance term (confounding), and noise
    # Note: We scale the nuisance term to ensure confounding is strong enough to bias naive results 
    confounding_effect = 1000 * nuisance_term
    
    Y = 2000 + (TRUE_EFFECT * T) + confounding_effect + np.random.normal(0,
    200, n)
    
    return X, T, Y, TRUE_EFFECT

In [11]:
if __name__ == "__main__":
    print("=========================================================")
    print("RARE DISEASE SIMULATION STUDY (N=500, P=100)")
    print("Features: High Dimensionality, Sparsity, Non-Linearity")
    print("=========================================================\n")

RARE DISEASE SIMULATION STUDY (N=500, P=100)
Features: High Dimensionality, Sparsity, Non-Linearity



In [18]:
# --- 1. CONTINUOUS OUTCOME ---
X, T, Y, true_eff = generate_rare_disease_data(n=500, p=100, setting='continuous')
sim = CausalEstimators(X, T, Y, outcome_type='continuous')

print(f"Scenario A: Continuous Outcome (e.g., Total Pay)")
print(f"True Causal Effect: {true_eff:.2f}")
print("-" * 65)
print(f"{'Method':<30} | {'Estimate':<15} | {'Bias':<10}")
print("-" * 65)

results_continuous = {
"Naive (Unadjusted)": sim.run_naive(),
"Regression (Linear):": sim.run_regression_adjustment(),
"IPTW (Logistic PS)": sim.run_iptw(),
"PSM (1:1 Nearest Neighbor)": sim.run_psm(),
"DML (Gradient Boosting)": sim.run_dml()
}

for method, val in results_continuous.items():
    bias = val - true_eff
    print(f"{method:<30} | {val:<15.2f} | {bias:<10.2f}")
    
print("\n" + "="*65 + "\n")

Scenario A: Continuous Outcome (e.g., Total Pay)
True Causal Effect: 1000.00
-----------------------------------------------------------------
Method                         | Estimate        | Bias      
-----------------------------------------------------------------
Naive (Unadjusted)             | 1620.98         | 620.98    
Regression (Linear):           | 1457.88         | 457.88    
IPTW (Logistic PS)             | 1500.66         | 500.66    
PSM (1:1 Nearest Neighbor)     | 1428.08         | 428.08    
DML (Gradient Boosting)        | 1312.57         | 312.57    


